### Usando a implementação do modelo Word2Vec da biblioteca: gensim

In [ ]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import gensim as gensim
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.scripts import word2vec2tensor
from gensim.models.callbacks import CallbackAny2Vec
import re
from google.colab import drive

### Esse notebook Jupyter foi criado usando Google Colab. Essa linha apenas monta a sua pasta do drive para salvar os arquivos dos modelos gerados em uma pasta que você pode definir o nome nas linhas abaixo (TCCLG nesse caso)

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/My Drive/TCCLG

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TCCLG


### Essa parte do código carrega o csv das diferentes representações das palavras de um dicionário e faz um pequeno parsing, construindo também o conjunto de palavras que vão ser incorporadas nesse modelo

In [ ]:
df = pd.read_csv('etiquetas2.csv')
df = df.replace(':','', regex=True)
print(df.head())
restricted_word_set = df['palavra'].to_numpy()

      palavra  ... Unnamed: 4
0         aba  ...        NaN
1       ababa  ...        NaN
2      abaçai  ...        NaN
3      abaças  ...        NaN
4  abacateiro  ...        NaN

[5 rows x 5 columns]


### A função abaixo serve para restringir um "vocabulário" do word2vec. Nesse contexto, utilizamos isto para remover as silabas individuais geradas mais a frente para que o modelo retorne apenas palavras completas. Isso fica mais claro mais a frente.

In [ ]:
# AVISO: quando você cria o modelo pela primeira vez, a propriedade vectors_norm é igual a None.
# Essa propriedade só é atualizada com o valor dos vetores após o primeiro uso, no formato numpy.ndarray.
# Dessa forma, antes de usar a função abaixo, é necessário utilizar o modelo uma primeira vez para
# inicializar os valores. Basta chamar algo como most_similar("gato").
def restrict_w2v(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.wv.vocab)):
        word = w2v.wv.index2entity[i]
        vec = w2v.wv.vectors[i]
        vocab = w2v.wv.vocab[word]
        vec_norm = w2v.wv.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            new_vectors_norm.append(vec_norm)

    w2v.wv.vocab = new_vocab
    w2v.wv.vectors = np.array(new_vectors)
    w2v.wv.index2entity = new_index2entity
    w2v.wv.index2word = new_index2entity
    w2v.wv.vectors_norm = new_vectors_norm

### Isso é apenas uma classe de logger para que consigamos ver o progresso do treinamento do modelo. Isso é um parâmetro da função, passado como callback

In [ ]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

        
epoch_logger = EpochLogger()

# Modelos utilizando a representação das Etiquetas
---



### Modelo básico (Frases Simples):
#### Nesse modelo, cada "frase'' do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas. Exemplo: ["abacaxi", "a", "ba", "ca", "'xi"]

In [ ]:
df_etiquetas = df[['palavra', 'etiquetas']]
print(df_etiquetas.head())
print("")

      palavra                                          etiquetas
0         aba                          [(SN V)]3.[(SA O)(SN V)]0
1       ababa          [(SN V)]1.[(SA O)(SN V)]3.[(SA O)(SN V)]0
2      abaçai  [(SN V)]1.[(SA O)(SN V)]1.[(SA F)(SN V)]1.[(SN...
3      abaças    [(SN V)]1.[(SA O)(SN V)]3.[(SA F)(SN V)(SC F)]0
4  abacateiro  [(SN V)]1.[(SA O)(SN V)]1.[(SA O)(SN V)]1.[(SA...



#### O começo da "frase" é a palavra original e as "silabas" são concatenadas na frase para que similaridades possam ser encontradas em um nível silábico pelo Word2Vec. A ideia aqui é que palavras com silabas semelhantes devem ter um alto nível de similaridade pelo modelo. Como as silabas aparecendo "proximas" das palavras na frase, essa semelhança é identificada até certo ponto pelo Word2Vec.

In [ ]:
corpus_etiquetas_basico = []
for index, row in df_etiquetas.iterrows():
  sentence = [row['palavra']]
  for syllable in row['etiquetas'].split('.'):
    sentence.append(syllable)
  corpus_etiquetas_basico.append(sentence)

In [ ]:
for k, v in enumerate(corpus_etiquetas_basico[:2]):
  print(f'{k}: {v}')

0: ['aba', '[(SN V)]3', '[(SA O)(SN V)]0']
1: ['ababa', '[(SN V)]1', '[(SA O)(SN V)]3', '[(SA O)(SN V)]0']


#### Após montar a representação, treinamos o modelo com os parâmetros definidos abaixo após multiplos testes. O modelo foi treinado por 30 iterações.

In [ ]:
model_etiquetas_basico = Word2Vec(corpus_etiquetas_basico, min_count=1, workers=8, window=10, sg=1, size=300, iter=30, callbacks=[epoch_logger])

# AVISO: esse é o print de primeiro uso mencionado anteriormente para inicializar os valores necessários para a função restrict_w2v
print(model_etiquetas_basico.wv.most_similar('abaçai'))

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
[('abiai', 0.9402936697006226), ('apiloada', 0.9391199946403503), ('itapui', 0.9376623630523682), ('aoba', 0.93554037809

#### Por fim, restringimos o vocabulário do modelo para remover as silabas. Dessa forma, as semelhanças encontradas são apenas com as palavras completas das frases geradas acima.

In [ ]:
restrict_w2v(model_etiquetas_basico, restricted_word_set)

#### Alguns exemplos dos resultados obtidos de coeficientes de similaridades para diferentes palavras. É facil notar a similaridade de algumas destas:

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_etiquetas_basico.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_etiquetas_basico.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_etiquetas_basico.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
catupecu: 0.9601438045501709
desoxirribose: 0.9584222435951233
desatualizadas: 0.9570872783660889
izanagi: 0.9541311264038086
rechaçada: 0.9533678293228149

Most similar to gue:
cortex: 0.8424830436706543
zenaida: 0.8289408683776855
medardo: 0.8264645338058472
locator: 0.8219625353813171
fumeiro: 0.8215623497962952

Most similar to grisalha:
xilogravura: 0.9669815301895142
cresimiro: 0.9540324211120605
parabolicas: 0.9483271837234497
comprometera: 0.9478045701980591
carboxilicos: 0.9450011253356934


#### Essa linha salva o modelo na pasta local. Caso esteja usando o colab, salvará na pasta definida nas primeiras linhas desse documento.

In [ ]:
model_etiquetas_basico.wv.save_word2vec_format("model_etiquetas_basico.model", binary=True)

#### Essa linha salva o arquivo do formato tensorflow que pode ser utilizado com o modelo acima em https://projector.tensorflow.org/ para uma visualização dos dados em forma de gráfico 3D com clusters de palavras similares.

In [ ]:
word2vec2tensor.word2vec2tensor("model_etiquetas_basico.model", "tensor-model-etiquetas-basico", True)

### Modelo Contagem (Frases com contador relativo a silaba)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas, com silabas duplicadas sendo diferenciadas por um contador utilizado como prefixo. Essa estratégia garante que o modelo não seja influenciado no aprendizado a acreditar que palavras que possuem silabas similares duplicadas possuem um grau de similaridade maior. Exemplo: ["anarcocomunismo", "0-a", "0-nar", "0-co", "1-co", "0-mu", "0-'nis", "0-mo"]


In [ ]:
corpus_etiquetas_contagem = []
for index, row in df_etiquetas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  for syllable in row['etiquetas'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    sentence.append(modified_syllable)
  corpus_etiquetas_contagem.append(sentence)

In [ ]:
for k, v in enumerate(corpus_etiquetas_contagem[:3]):
  print(f'{k}: {v}')

0: ['aba', '0-[(SN V)]3', '0-[(SA O)(SN V)]0']
1: ['ababa', '0-[(SN V)]1', '0-[(SA O)(SN V)]3', '0-[(SA O)(SN V)]0']
2: ['abaçai', '0-[(SN V)]1', '0-[(SA O)(SN V)]1', '0-[(SA F)(SN V)]1', '0-[(SN V)]3']


In [ ]:
model_etiquetas_contagem = Word2Vec(corpus_etiquetas_contagem, min_count=1, workers=8, window=30, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_etiquetas_contagem.wv.most_similar('abaçai'))

Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end
Epoch #34 start
Epoch #34 end
Epoch #35 start
Epoch #35 end
Epoch #36 start
Epoch #36 end
Epoch #37 start
Epoch #37 end
Epoch #38 start
Epoch #38 end
Epoch #39 start
Epoch #39 end
Epoch #40 start
Epoch #40 end
Epoch #41 start
Epoch #41 end
Epoch #42 start
Epoch #42 end
Epoch #43 start
Epoch #43 end
Epoch #44 start
Epoch #44 end
Epoch #45 start
Epoch #45 end
Epoch #46 start
Epoch #46 end
Epoch #47 start
Epoch #47 end
Epoch #48 start
Epoch #48 end
Epoch #49 start
Epoch #49 end
Epoch #50 start
Epoch #50 end
Epoch #51 start
Epoch #51 end
Epoch #52 start
Epoch #52 end
Epoch #53 start
Epoch #53 end
Epoch #54 start
Epoch #54 end
Epoch #55 start
Epoch #55 end
Epoch #56 start
Epoch #56 end
Epoch #57 start
Epoch #57 end
Epoch #58 start
Epoch #58 end
Epoch #59 start
Epoch #59 end
[('averiguadas', 0.95344078540802), ('semi-acabados', 0.9494262337684631), ('uruçui', 0.947746634483

In [ ]:
restrict_w2v(model_etiquetas_contagem, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_etiquetas_contagem.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_etiquetas_contagem.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_etiquetas_contagem.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_etiquetas_contagem.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
papicu: 0.9401211738586426
cocorico: 0.9352646470069885
ibirapuera: 0.9300083518028259
debatida: 0.9294928312301636
cabinete: 0.9223779439926147

Most similar to gue:
gago: 0.8835179805755615
pobeda: 0.8729716539382935
jucu: 0.8653994798660278
tete: 0.854726254940033
jacu: 0.8478527069091797

Most similar to grisalha:
clavatula: 0.9660098552703857
prefira: 0.9589757323265076
triguilho: 0.9575575590133667
plaquetario: 0.9523038268089294
abriga-la: 0.9499630928039551

Most similar to sintetizado:
sindicalizada: 0.970808207988739
desemboca: 0.9689019918441772
santidade: 0.9686051607131958
sintetizadas: 0.9639430046081543
simpatiza: 0.9628968238830566


In [ ]:
model_etiquetas_contagem.wv.save_word2vec_format("model_etiquetas_contagem.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_etiquetas_contagem.model", "tensor-model-etiquetas-contagem", True)

### Modelo Chunks (frases com sub-palavras)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas. Além disso, são geradas todas sub-palavras possíveis com as silabas levando em consideração a ordem das mesmas na palavra original. Essa estratégia tem como objetivo tentar influenciar o aprendizado a detectar similaridade de contexto com base em sub-palavras que podem ocorrer dentro de palavras maiores, similarmente a estrategia do FastText mas considerando silabas ao invés de n-grams. Exemplo: ["abacaxi", "a", "ba", "ca", "'xi", "aba", "baca", "ca'xi", "abaca", "baca'xi", "abaca'xi"]


In [ ]:
corpus_etiquetas_chunk = []
for index, row in df_etiquetas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['etiquetas'].split('.'):
    modified_syllable = f'{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunk}")
        chunks.append(chunk)
  corpus_etiquetas_chunk.append(sentence)

In [ ]:
for k, v in enumerate(corpus_etiquetas_chunk[:3]):
  print(f'{k}: {v}')

0: ['aba', '[(SN V)]3', '[(SA O)(SN V)]0', '[(SN V)]3 [(SA O)(SN V)]0']
1: ['ababa', '[(SN V)]1', '[(SA O)(SN V)]3', '[(SA O)(SN V)]0', '[(SN V)]1 [(SA O)(SN V)]3', '[(SA O)(SN V)]3 [(SA O)(SN V)]0', '[(SN V)]1 [(SA O)(SN V)]3 [(SA O)(SN V)]0']
2: ['abaçai', '[(SN V)]1', '[(SA O)(SN V)]1', '[(SA F)(SN V)]1', '[(SN V)]3', '[(SN V)]1 [(SA O)(SN V)]1', '[(SA O)(SN V)]1 [(SA F)(SN V)]1', '[(SA F)(SN V)]1 [(SN V)]3', '[(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]1', '[(SA O)(SN V)]1 [(SA F)(SN V)]1 [(SN V)]3', '[(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]1 [(SN V)]3']


In [ ]:
model_etiquetas_chunk = Word2Vec(corpus_etiquetas_chunk, min_count=1, workers=8, window=50, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_etiquetas_chunk.wv.most_similar('abaçai'))

Epoch #60 start
Epoch #60 end
Epoch #61 start
Epoch #61 end
Epoch #62 start
Epoch #62 end
Epoch #63 start
Epoch #63 end
Epoch #64 start
Epoch #64 end
Epoch #65 start
Epoch #65 end
Epoch #66 start
Epoch #66 end
Epoch #67 start
Epoch #67 end
Epoch #68 start
Epoch #68 end
Epoch #69 start
Epoch #69 end
Epoch #70 start
Epoch #70 end
Epoch #71 start
Epoch #71 end
Epoch #72 start
Epoch #72 end
Epoch #73 start
Epoch #73 end
Epoch #74 start
Epoch #74 end
Epoch #75 start
Epoch #75 end
Epoch #76 start
Epoch #76 end
Epoch #77 start
Epoch #77 end
Epoch #78 start
Epoch #78 end
Epoch #79 start
Epoch #79 end
Epoch #80 start
Epoch #80 end
Epoch #81 start
Epoch #81 end
Epoch #82 start
Epoch #82 end
Epoch #83 start
Epoch #83 end
Epoch #84 start
Epoch #84 end
Epoch #85 start
Epoch #85 end
Epoch #86 start
Epoch #86 end
Epoch #87 start
Epoch #87 end
Epoch #88 start
Epoch #88 end
Epoch #89 start
Epoch #89 end
[('itajai', 0.9672127962112427), ('iguaçuana', 0.9357708692550659), ('teodosiana', 0.934071302413940

In [ ]:
restrict_w2v(model_etiquetas_chunk, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_etiquetas_chunk.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
jequitiba: 0.9864708185195923
jaboticaba: 0.9369639754295349
caqueta: 0.9348481297492981
jucutuquara: 0.930809497833252
jabuticaba: 0.9307656288146973

Most similar to gue:
ta: 0.9034301042556763
gui: 0.8916037082672119
ga: 0.8886886835098267
cu: 0.884696900844574
gua: 0.8834198713302612

Most similar to grisalha:
groselha: 0.9883201122283936
grisalho: 0.987287163734436
crucero: 0.9848392605781555
prefiro: 0.9837859272956848
crassula: 0.9831794500350952

Most similar to sintetizado:
sintetizada: 0.9917088747024536
batizado: 0.9406454563140869
cobiçada: 0.940295934677124
cativada: 0.9375988245010376
deduzido: 0.9363431334495544

Most similar to aba:
uca: 0.9801416397094727
ago: 0.980124831199646
ude: 0.9795656204223633
upe: 0.9793827533721924
ugo: 0.9788247346878052

Most similar to bata:
coba: 0.9675712585449219
bobu: 0.9675328731536865
babu: 0.9657381772994995
tabi: 0.9650372266769409
cupe: 0.9648560285568237

Most similar to bantu:
ponda: 0.9746549129486084

In [ ]:
model_etiquetas_chunk.wv.save_word2vec_format("model_etiquetas_chunk.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_etiquetas_chunk.model", "tensor-etiquetas-chunk", True)

### Contagem + Chunks (frases com sub-palavras e contador)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas,  com silabas duplicadas sendo diferenciadas por um contador utilizado como prefixo. Além disso, são geradas todas sub-palavras possíveis com as silabas levando em consideração a ordem das mesmas na palavra original. As sub-palavras também possuem um prefixo de contador de número de ocorrências. O objetivo é combinar as duas estratégias anteriores e obter as vantagens de ambas estratégias. Exemplo: ["abacaxi", "0-a", "0-ba", "0-ca", "0-'xi", "0\_aba", "0\_baca", "0\_ca'xi", "0\_abaca", "0\_baca'xi", "0\_abaca'xi"]

In [ ]:
corpus_etiquetas_contagem_chunk = []
for index, row in df_etiquetas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['etiquetas'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
    # sentence.append(row['palavra'])
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunks.count(chunk)}_{chunk}")
        chunks.append(chunk)
        # sentence.append(row['palavra'])
  if 'pongo' in sentence:
    print(sentence)
  corpus_etiquetas_contagem_chunk.append(sentence)

['pongo', '0-[(SA O)(SN V)(SC N)]3', '0-[(SA O)(SN V)]0', '0_[(SA O)(SN V)(SC N)]3 [(SA O)(SN V)]0']


In [ ]:
for k, v in enumerate(corpus_etiquetas_contagem_chunk[:3]):
  print(f'{k}: {v}')

0: ['aba', '0-[(SN V)]3', '0-[(SA O)(SN V)]0', '0_[(SN V)]3 [(SA O)(SN V)]0']
1: ['ababa', '0-[(SN V)]1', '0-[(SA O)(SN V)]3', '0-[(SA O)(SN V)]0', '0_[(SN V)]1 [(SA O)(SN V)]3', '0_[(SA O)(SN V)]3 [(SA O)(SN V)]0', '0_[(SN V)]1 [(SA O)(SN V)]3 [(SA O)(SN V)]0']
2: ['abaçai', '0-[(SN V)]1', '0-[(SA O)(SN V)]1', '0-[(SA F)(SN V)]1', '0-[(SN V)]3', '0_[(SN V)]1 [(SA O)(SN V)]1', '0_[(SA O)(SN V)]1 [(SA F)(SN V)]1', '0_[(SA F)(SN V)]1 [(SN V)]3', '0_[(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]1', '0_[(SA O)(SN V)]1 [(SA F)(SN V)]1 [(SN V)]3', '0_[(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]1 [(SN V)]3']


In [ ]:
model_etiquetas_contagem_chunk = Word2Vec(corpus_etiquetas_contagem_chunk, min_count=1, workers=8, window=50, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_etiquetas_contagem_chunk.wv.most_similar('abaçai'))

Epoch #60 start
Epoch #60 end
Epoch #61 start
Epoch #61 end
Epoch #62 start
Epoch #62 end
Epoch #63 start
Epoch #63 end
Epoch #64 start
Epoch #64 end
Epoch #65 start
Epoch #65 end
Epoch #66 start
Epoch #66 end
Epoch #67 start
Epoch #67 end
Epoch #68 start
Epoch #68 end
Epoch #69 start
Epoch #69 end
Epoch #70 start
Epoch #70 end
Epoch #71 start
Epoch #71 end
Epoch #72 start
Epoch #72 end
Epoch #73 start
Epoch #73 end
Epoch #74 start
Epoch #74 end
Epoch #75 start
Epoch #75 end
Epoch #76 start
Epoch #76 end
Epoch #77 start
Epoch #77 end
Epoch #78 start
Epoch #78 end
Epoch #79 start
Epoch #79 end
Epoch #80 start
Epoch #80 end
Epoch #81 start
Epoch #81 end
Epoch #82 start
Epoch #82 end
Epoch #83 start
Epoch #83 end
Epoch #84 start
Epoch #84 end
Epoch #85 start
Epoch #85 end
Epoch #86 start
Epoch #86 end
Epoch #87 start
Epoch #87 end
Epoch #88 start
Epoch #88 end
Epoch #89 start
Epoch #89 end
[('itajai', 0.9709231853485107), ('otaviano', 0.925238847732544), ('iguaçuano', 0.9249710440635681),

In [ ]:
wordsThatMatch = model_etiquetas_contagem_chunk.wv.most_similar("0_[(SA O)(SN V)]1 [(SA O)(SN V)]1 [(SA O)(CN1 V)(CN2 V)]3 [(SA L)(SN V)]0")
print([x for x in wordsFromContext if '[' not in x[0]])

[('abacateiro', 0.0020342753), ('abatedouro', 0.0018883646), ('bebedouro', 0.0013970652), ('bebedeira', 0.0013948699), ('bacabeira', 0.0012347283), ('batuqueiro', 0.0011780391)]


In [ ]:
wordsFromContext = model_etiquetas_contagem_chunk.predict_output_word(['0_[(SA O)(SN V)]1 [(SA O)(SN V)]1 [(SA O)(CN1 V)(CN2 V)]3 [(SA L)(SN V)]0'], topn = 50)
print([x for x in wordsFromContext if '[' not in x[0]])

[('abacateiro', 0.0020342753), ('abatedouro', 0.0018883646), ('bebedouro', 0.0013970652), ('bebedeira', 0.0013948699), ('bacabeira', 0.0012347283), ('batuqueiro', 0.0011780391)]


In [ ]:
wordsFromContext2 = model_etiquetas_contagem_chunk.predict_output_word(['0-[(SA O)(SN V)]1', '1-[(SA O)(SN V)]1', '0-[(SA O)(SN V)]3', '0-[(SA L)(SN V)]0'], topn = 1000)
print([x for x in wordsFromContext2 if '[' not in x[0]])

[('estatutarias', 0.00019087568), ('dedicatorias', 0.00016623239), ('estatutarios', 0.00016141053), ('jucutuquara', 0.00015894814), ('multiarquitetura', 0.00015757912), ('estatutario', 0.00015424627), ('estatutaria', 0.00015175845), ('escapatoria', 0.00014809943), ('dedicatoria', 0.00014739257), ('pseudopaludicola', 0.00014591451), ('interrogatorios', 0.00014209916), ('interrogatorio', 0.00013795495), ('discotecario', 0.00013639478), ('recapitula', 0.00013593843), ('arquitetura', 0.00013516343), ('repetidora', 0.00013302271), ('judicatura', 0.00013247466), ('peditorios', 0.00013181636), ('investigatorio', 0.00012990614), ('educadora', 0.00012962511), ('catabolica', 0.00012956098), ('categorico', 0.00012645447), ('categorica', 0.00012572811), ('pitagorica', 0.00012537821), ('comunicadora', 0.00012469926), ('intimidadora', 0.00012384518), ('desembocadura', 0.00012199584), ('anti-catolicos', 0.000119895325), ('desestabilizadora', 0.000119793156), ('anticatolica', 0.0001196563), ('boticari

In [ ]:
wordsFromContext2 = model_etiquetas_contagem_chunk.predict_output_word(['0-[(SA O)(SN V)]1', '1-[(SA O)(SN V)]1', '0-[(SA O)(SN V)]3', '0-[(SA L)(SN V)]0'], topn = 1000)
print([x for x in wordsFromContext2 if '[' not in x[0] and 'bebedouro' in x[0]])

[]


In [ ]:
wordsFromContext2 = model_etiquetas_contagem_chunk.predict_output_word(['0-[(SA O)(SN V)]1', '1-[(SA O)(SN V)]1', '0-[(SA L)(SN V)]0'], topn = 1000)
print([x for x in wordsFromContext2 if '[' not in x[0] and 'bebedouro' in x[0]])

[('bebedouro', 0.00013084135)]


In [ ]:
wordsFromContext2 = model_etiquetas_contagem_chunk.predict_output_word(['0-[(SA O)(SN V)]1', '1-[(SA O)(SN V)]1', '0-[(SA O)(SN V)]3', '0-[(SA L)(SN V)]0', '0_[(SA O)(SN V)]1 [(SA O)(SN V)]1'], topn = 1000)
print([x for x in wordsFromContext2 if '[' not in x[0] and 'bebedouro' in x[0]])

[('bebedouro', 7.987386e-05)]


In [ ]:
restrict_w2v(model_etiquetas_contagem_chunk, restricted_word_set)

In [ ]:
print(model_etiquetas_contagem_chunk.predict_output_word(['0-[(SA O)(SN V)(SC N)]3', '0-[(SA O)(SN V)]0'], topn = 100))

[('0_[(SA O)(SN V)(SC N)]3 [(SA O)(SN V)]0', 0.019534132), ('0-[(SA O)(SN V)(SC N)]3', 0.00870735), ('pretenda', 0.0012843164), ('mordente', 0.0012532776), ('pongo', 0.0011875498), ('mordendo', 0.0011658141), ('pungente', 0.0011428068), ('rebento', 0.0010280637), ('pumba', 0.0009878356), ('ponto', 0.0009720551), ('penta', 0.0009592839), ('propinqua', 0.0009332629), ('punta', 0.00092579756), ('redunda', 0.0009226185), ('probando', 0.0009189746), ('reerguendo', 0.0009139765), ('multibanco', 0.00090489193), ('pretendente', 0.0008900375), ('pingo', 0.0008882174), ('quendo', 0.00087069266), ('ponta', 0.00086608453), ('punto', 0.0008536273), ('0_[(SA F)(SN V)(SC O)]1 [(SA O)(SN V)(SC N)]3 [(SA O)(SN V)]0', 0.00085262425), ('prudente', 0.0008319306), ('obstante', 0.00082624453), ('prorrogando', 0.00081343716), ('opondo', 0.0008128866), ('oconto', 0.0007998834), ('portante', 0.00079466973), ('0_[(SA O)(SN V)(SC N)]3 [(SA O)(SN V)]0 [(CA1 O)(CA2 L)(SN V)]0', 0.0007908225), ('progredindo', 0.000

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to pando:')
for k, v in enumerate(model_etiquetas_contagem_chunk.wv.most_similar('pando')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
jequitiba: 0.939752459526062
jaboticaba: 0.8923256397247314
jabuticaba: 0.8908146619796753
0_[(SA F)(SN V)]1 [(SA O)(SN V)]1 [(SA O)(SN V)]1 [(SA O)(SN V)]3 [(SA O)(SN V)]0: 0.8815503716468811
judicativa: 0.8590869903564453

Most similar to gue:
ga: 0.9541316032409668
gui: 0.9506388902664185
go: 0.9497466087341309
gua: 0.9283155798912048
do: 0.8460502624511719

Most similar to grisalha:
grisalho: 0.9811973571777344
groselha: 0.9718570709228516
gravilha: 0.9574096202850342
glafira: 0.9358783960342407
escreve-lo: 0.9047542810440063

Most similar to sintetizado:
0_[(SA F)(SN V)(SC N)]1 [(SA O)(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]3 [(SA O)(SN V)]0: 0.9765576720237732
sintetizada: 0.9643033146858215
0_[(SA F)(SN V)(SC N)]1 [(SA O)(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]3: 0.8889876008033752
0_[(SA F)(SN V)(SC N)]1 [(SA O)(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]3 [(SA O)(SN V)(SC F)]0: 0.8822759389877319
sintetizadas: 0.8784647583961487

Most similar to aba:
upe: 0

In [ ]:
model_etiquetas_contagem_chunk.wv.save_word2vec_format("model_etiquetas_contagem_chunk.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_etiquetas_contagem_chunk.model", "tensor-etiquetas-contagem-chunk", True)

### FastText - Contagem + Chunks
#### Esse modelo acabou sendo discutido apenas brevemente no TCC, mas é a mesma representação acima sendo utilizada para treinar outro modelo, o FastText, que diferentemente do Word2Vec, tem por padrão a funcionalidade de identificar similaridades em silabas sem necessidade da inclusão delas na "frase". Apesar disso, a detecção de silabas do FastText é implementada pela utilização de n-grams, que são pequenas parcelas da palavra de tamanhos variados. Isso não é bem preciso para as nossas representações de etiquetagem e de fonética, visto que existem símbolos que quando removidos tornam completamente diferente a representação da palavra. Apesar disso, foi decidido tentar testar os resultados obtidos com esse outro modelo para comparação com o Word2Vec.

In [ ]:
corpus_etiquetas_fast_text_cc = []
for index, row in df_etiquetas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['etiquetas'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunks.count(chunk)}_{chunk}")
        chunks.append(chunk)
  corpus_etiquetas_fast_text_cc.append(sentence)

In [ ]:
for k, v in enumerate(corpus_etiquetas_fast_text_cc[:3]):
  print(f'{k}: {v}')

0: ['aba', '0-[(SN V)]3', '0-[(SA O)(SN V)]0', '0_[(SN V)]3 [(SA O)(SN V)]0']
1: ['ababa', '0-[(SN V)]1', '0-[(SA O)(SN V)]3', '0-[(SA O)(SN V)]0', '0_[(SN V)]1 [(SA O)(SN V)]3', '0_[(SA O)(SN V)]3 [(SA O)(SN V)]0', '0_[(SN V)]1 [(SA O)(SN V)]3 [(SA O)(SN V)]0']
2: ['abaçai', '0-[(SN V)]1', '0-[(SA O)(SN V)]1', '0-[(SA F)(SN V)]1', '0-[(SN V)]3', '0_[(SN V)]1 [(SA O)(SN V)]1', '0_[(SA O)(SN V)]1 [(SA F)(SN V)]1', '0_[(SA F)(SN V)]1 [(SN V)]3', '0_[(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]1', '0_[(SA O)(SN V)]1 [(SA F)(SN V)]1 [(SN V)]3', '0_[(SN V)]1 [(SA O)(SN V)]1 [(SA F)(SN V)]1 [(SN V)]3']


In [ ]:
model_etiquetas_fast_text_cc = FastText(corpus_etiquetas_fast_text_cc, min_count=1, workers=8, window=10, sg=1, size=300, iter=30, min_n=2, max_n=2, callbacks=[epoch_logger])
print(model_etiquetas_fast_text_cc.wv.most_similar('simtetizando'))

Epoch #120 start
Epoch #120 end
Epoch #121 start
Epoch #121 end
Epoch #122 start
Epoch #122 end
Epoch #123 start
Epoch #123 end
Epoch #124 start
Epoch #124 end
Epoch #125 start
Epoch #125 end
Epoch #126 start
Epoch #126 end
Epoch #127 start
Epoch #127 end
Epoch #128 start
Epoch #128 end
Epoch #129 start
Epoch #129 end
Epoch #130 start
Epoch #130 end
Epoch #131 start
Epoch #131 end
Epoch #132 start
Epoch #132 end
Epoch #133 start
Epoch #133 end
Epoch #134 start
Epoch #134 end
Epoch #135 start
Epoch #135 end
Epoch #136 start
Epoch #136 end
Epoch #137 start
Epoch #137 end
Epoch #138 start
Epoch #138 end
Epoch #139 start
Epoch #139 end
Epoch #140 start
Epoch #140 end
Epoch #141 start
Epoch #141 end
Epoch #142 start
Epoch #142 end
Epoch #143 start
Epoch #143 end
Epoch #144 start
Epoch #144 end
Epoch #145 start
Epoch #145 end
Epoch #146 start
Epoch #146 end
Epoch #147 start
Epoch #147 end
Epoch #148 start
Epoch #148 end
Epoch #149 start
Epoch #149 end
[('sintetizando', 0.9465638399124146), (

In [ ]:
restrict_w2v(model_etiquetas_fast_text_cc, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_etiquetas_fast_text_cc.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
guapituba: 0.9010206460952759
catagua: 0.8873807191848755
capita: 0.8774635791778564
bodopita: 0.877284824848175
guatape: 0.874596118927002

Most similar to gue:
guede: 0.9104888439178467
gua: 0.9040183424949646
guegue: 0.9022629857063293
guape: 0.8908253908157349
que: 0.8709769248962402

Most similar to grisalha:
grisalho: 0.9836864471435547
clarissa: 0.9152013659477234
para-brisa: 0.9027152061462402
brilhosa: 0.8950402140617371
gravilha: 0.8917633891105652

Most similar to sintetizado:
sintetizada: 0.9878792762756348
sintetiza: 0.9286132454872131
sintetizando: 0.8990702629089355
santificado: 0.8953001499176025
santificada: 0.8805282711982727

Most similar to aba:
apa: 0.959701418876648
aca: 0.9589343667030334
apca: 0.9509650468826294
aga: 0.9466910362243652
ata: 0.941753625869751

Most similar to bata:
bacata: 0.9524035453796387
baga: 0.9449193477630615
beta: 0.941749095916748
batota: 0.941179633140564
baba: 0.9406967163085938

Most similar to bantu:
cantu:

In [ ]:
print(model_etiquetas_fast_text_cc.wv.most_similar('simtetizando'))
print(model_etiquetas_fast_text_cc.wv.most_similar('facista'))
print(model_etiquetas_fast_text_cc.wv.most_similar('facistas'))
print(model_etiquetas_fast_text_cc.wv.most_similar('panda'))
print(model_etiquetas_fast_text_cc.wv.most_similar('pamda'))
print(model_etiquetas_fast_text_cc.wv.most_similar('caxorro'))
print(model_etiquetas_fast_text_cc.wv.most_similar('bantu'))

[('sintetizando', 0.9401115775108337), ('simpatizando', 0.9323481321334839), ('batizando', 0.9038254022598267), ('sintetizado', 0.8970606327056885), ('tematizando', 0.8929797410964966), ('sintetiza', 0.8915205597877502), ('simpatizante', 0.8900021314620972), ('quantizado', 0.8770200610160828), ('sintetizada', 0.8733042478561401), ('gozando', 0.8649653196334839)]
[('cista', 0.9275723695755005), ('sufista', 0.923670768737793), ('sofista', 0.9214907288551331), ('vista', 0.9126813411712646), ('pacifista', 0.9075833559036255), ('sista', 0.9042036533355713), ('vistosa', 0.9031184911727905), ('xisto', 0.8987702131271362), ('musicista', 0.8977652788162231), ('cisto', 0.8939428329467773)]
[('cistas', 0.9425678253173828), ('fascistas', 0.9424934387207031), ('sexistas', 0.9408373832702637), ('sufistas', 0.93963223695755), ('sofistas', 0.9378201961517334), ('vistas', 0.9314824342727661), ('xistos', 0.9268124103546143), ('pacifistas', 0.9245144128799438), ('vistosas', 0.9244798421859741), ('cistos'

In [ ]:
print(model_etiquetas_fast_text_cc.wv.most_similar('bebedouro'))
print(model_etiquetas_fast_text_cc.wv.most_similar('bebedoro'))
#[('abatedouro', 0.9132556915283203), ('bebedeira', 0.9124612212181091), ('bacabeira', 0.9043855667114258), ('batuqueiro', 0.895446240901947), ('batedeira', 0.893902063369751), ('tabaqueira', 0.8845760822296143), ('abacateiro', 0.8816585540771484), ('puteiro', 0.8772128820419312), ('cauduro', 0.8724573850631714), ('bobeira', 0.8722970485687256)]
#[('batedora', 0.9272826313972473), ('cupedora', 0.8930730223655701), ('cabedelo', 0.8806620836257935), ('botupora', 0.8774875402450562), ('bebedo', 0.877417802810669), ('pedatura', 0.8731216192245483), ('tutorado', 0.8725395202636719), ('gorutuba', 0.8688998222351074), ('bagatela', 0.8665062189102173), ('garopaba', 0.863147497177124)]

[('abatedouro', 0.9117780923843384), ('bebedeira', 0.9081835150718689), ('bacabeira', 0.8993093967437744), ('batedeira', 0.8945928812026978), ('batuqueiro', 0.8913049697875977), ('tabaqueira', 0.8821417689323425), ('beirado', 0.8780285120010376), ('depauperado', 0.8774333000183105), ('bagageiro', 0.8716026544570923), ('cauduro', 0.8715219497680664)]
[('batedora', 0.9260560274124146), ('cupedora', 0.8971521854400635), ('tutorado', 0.8857333660125732), ('cabedelo', 0.885244607925415), ('botupora', 0.8833124041557312), ('bebedo', 0.8808751106262207), ('boreto', 0.8796001076698303), ('tocadora', 0.8761911988258362), ('cabeludo', 0.8737738132476807), ('tutelado', 0.8668221235275269)]


In [ ]:
model_etiquetas_fast_text_cc.wv.save_word2vec_format("model_etiquetas_fast_text_cc.model", binary=False)
word2vec2tensor.word2vec2tensor("model_etiquetas_fast_text_cc.model", "tensor-model-etiquetas-fast-text-cc", False)